In [22]:
import pandas as pd
from utils import *
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import numpy as np

In [5]:
import pickle
with open('../data/features.pickle', 'rb') as handle:
    df = pickle.load(handle)[xnames[0: 5]]

In [16]:
n_values = df.max() + 1

In [12]:
X = df.sample(frac=0.01, random_state=2333).astype(int).values

In [18]:
enc = OneHotEncoder(n_values=n_values, categorical_features='all')

In [19]:
X_oh = enc.fit_transform(X)  

In [24]:
X_train, X_test = train_test_split(X_oh, test_size=0.1)

In [25]:
from keras.layers import Input, Dense
from keras.models import Model
from keras import regularizers

# this is the size of our encoded representations
encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
inputs = Input(shape=(X_oh.shape[1],))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu')(inputs)
#encoded = Dense(encoding_dim, activation='relu', activity_regularizer=regularizers.l1(10e-5))(inputs)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(X_oh.shape[1], activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(inputs, decoded)

# this model maps an input to its encoded representation
encoder = Model(inputs, encoded)

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

/home/ubuntu/workspace/tools/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [26]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [27]:
autoencoder.fit(X_train, X_train,
                epochs=1,
                batch_size=256,
                shuffle=True,
                validation_data=(X_test, X_test))

Train on 1833249 samples, validate on 203695 samples
Epoch 1/1
1833249/1833249 [==============================] - 422s 230us/step - loss: 0.0705 - val_loss: 0.0036


In [33]:
from sklearn.externals import joblib
joblib.dump(enc, 'ohe.pkl') 

['ohe.pkl']

In [32]:
encoder.save('../model/encoder')